In [1]:
import numpy as np

In [3]:
class Softmaxreggression:
    def __init__(self,lr=0.01,niters=1000,l2_lambda=0.0):
        self.lr=lr
        self.niters=niters
        self.l2_lambda=l2_lambda
        self.w=None
        self.b=None
        self.loss_history=[]

    def softmax(self,z):
        expz=np.exp(z-np.max(z,axis=1,keepdims=True)) #expz=(m,k)
        return expz/np.sum(expz,axis=1,keepdims=True) #shape(m,k) 

    def fit(self,X,Y):
        nsamples,nfeatures=X.shape
        k=Y.shape[1]
        self.w=np.zeros((nfeatures,k))
        self.b=np.zeros((1,k))


        for _ in range(self.niters):

            z=X@self.w + self.b  # z=(m,k)

            y_pred=self.softmax(z)  #ypred=(m,k)

            costf=-np.sum(Y*np.log(y_pred))/nsamples  + self.l2_lamda*np.sum(self.w**2)/(2*nsamples)
            self.loss_history.append(costf)

            dw=X.T@(y_pred-Y)/nsamples + self.l2_lamda/nsamples*self.w  #dw=(n,k)
            db=np.sum(y_pred-Y,axis=0,keepdims=True)/nsamples  #db=(1,k)

            self.w-=self.lr*dw
            self.b-=self.lr*db

    def predict(self,X):
        y_pred=self.softmax(X@self.w + self.b)
        return y_pred


In [8]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [1]:
import numpy as np
from scipy.optimize import minimize

def compute_ratios(prices, avg_price):
    n = len(prices)
    
    def loss(q):
        weighted_avg = np.dot(prices, q) / np.sum(q)
        return (weighted_avg - avg_price)**2

    initial_guess = np.ones(n)
    bounds = [(0.01, 100) for _ in range(n)]  # avoid zero quantities

    result = minimize(loss, initial_guess, bounds=bounds)
    
    if result.success:
        q = result.x
        q_normalized = q / q[0]  # Normalize based on first quantity
        return q_normalized
    else:
        raise ValueError("Optimization failed")


In [9]:
ratios=compute_ratios([13.60,13.75,13.90,14,14.20],14.1)
ratios

array([  1.        ,   1.        ,  55.10527697, 114.42264641,
       233.05627085])

In [11]:
import itertools

def find_realistic_ratios(prices, target_avg, max_units=20, tolerance=0.01):
    n = len(prices)
    best_match = None
    best_diff = float('inf')

    for q in itertools.product(range(1, max_units+1), repeat=n):
        total_qty = sum(q)
        weighted_sum = sum(p * qty for p, qty in zip(prices, q))
        avg = weighted_sum / total_qty
        diff = abs(avg - target_avg)

        if diff < tolerance:
            if diff < best_diff:
                best_match = q
                best_diff = diff

    return best_match


In [21]:
l=find_realistic_ratios([13.60,13.75,13.90,14,14.20],14.1)
l

(1, 2, 1, 1, 15)

In [23]:
import itertools

def find_ratios_to_target(prices, target_percent=0.993, max_units=15, tolerance=0.01):
    m = len(prices)
    target_avg = prices[-1] * target_percent
    best_ratio = None
    best_diff = float('inf')

    for ratio in itertools.product(range(1, max_units + 1), repeat=m):
        total_qty = sum(ratio)
        weighted_sum = sum(p * q for p, q in zip(prices, ratio))
        avg = weighted_sum / total_qty
        diff = abs(avg - target_avg)

        if diff < tolerance:
            if diff < best_diff:
                best_diff = diff
                best_ratio = ratio

    return best_ratio, best_diff


In [25]:
prices = [13.60, 13.75, 13.90, 14.00, 14.20]
ratio, diff = find_ratios_to_target(prices, target_percent=0.993)

print("Best Ratio:", ratio)
print("Achieved Average:", sum(p*q for p, q in zip(prices, ratio)) / sum(ratio))
print("Target Average:", 0.993 * prices[-1])
print("Difference:", diff)


Best Ratio: (1, 2, 1, 1, 15)
Achieved Average: 14.1
Target Average: 14.1006
Difference: 0.000600000000000378
